In [1]:
## Code for reading external HuggingFace token, if necessary
def get_huggingface_token():
    f = open("huggingface_token.txt", "r")
    return (f.read())

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Input list of texts
text_list = ["Text example one", "Text example two", "Another text"]

# Tokenize the list of texts
inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors='pt')

# Pass through the model
with torch.no_grad():  # Disable gradient calculation for efficiency
    outputs = model(**inputs)

# Extract embeddings (typically using the mean of the last hidden state)
embeddings = outputs.last_hidden_state.mean(dim=1)  # Shape: (batch_size, embedding_dim)

print(embeddings.shape)  # Should print (number_of_texts, embedding_dim)
print(embeddings)

torch.Size([3, 768])
tensor([[ 0.0513, -0.1666, -0.0242,  ..., -0.0026,  0.0142,  0.0259],
        [ 0.0836, -0.1754, -0.0299,  ...,  0.0095, -0.0670,  0.0262],
        [ 0.1726, -0.0736,  0.0352,  ...,  0.0499, -0.0935, -0.0759]])


In [8]:
print(len(outputs))

2


In [ ]:
import openai

openai.api_key = 'your-api-key'

response = openai.Embedding.create(
  input='Your input text here',
  model='text-embedding-ada-002'
)

In [5]:
texts = ["How do I get a replacement Medicare card?",
        "What is the monthly premium for Medicare Part B?",
        "How do I terminate my Medicare Part B (medical insurance)?",
        "How do I sign up for Medicare?",
        "Can I sign up for Medicare Part B if I am working and have health insurance through an employer?",
        "How do I sign up for Medicare Part B if I already have Part A?",
        "What are Medicare late enrollment penalties?",
        "What is Medicare and who can get it?",
        "How can I get help with my Medicare Part A and Part B premiums?",
        "What are the different parts of Medicare?",
        "Will my Medicare premiums be higher because of my higher income?",
        "What is TRICARE ?",
        "Should I sign up for Medicare Part B if I have Veterans' Benefits?"]

output = query(texts)
print(output)

{'error': '401 Unauthorized'}


In [ ]:
import pandas as pd
embeddings = pd.DataFrame(output)